<a href="https://colab.research.google.com/github/blowmeaway1234/Artificial-Intelligence/blob/main/Midterm_Vietnam_Current_Recognition_build_with_ANN_using_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# Load dataset Vietnamese Currency 

In [ ]:
!ls gdrive/MyDrive/DataSetCurrency # test data 

ls: cannot access 'gdrive/MyDrive/DataSetCurrent': No such file or directory


In [ ]:
# Unzip Test and Train data
! unzip gdrive/MyDrive/DataSetVNC/testingSet.zip
! unzip gdrive/MyDrive/DataSetVNC/trainingSet.zip

unzip:  cannot find or open gdrive/MyDrive/DataSetVNC/testingSet.zip, gdrive/MyDrive/DataSetVNC/testingSet.zip.zip or gdrive/MyDrive/DataSetVNC/testingSet.zip.ZIP.
unzip:  cannot find or open gdrive/MyDrive/DataSetVNC/trainingSet.zip, gdrive/MyDrive/DataSetVNC/trainingSet.zip.zip or gdrive/MyDrive/DataSetVNC/trainingSet.zip.ZIP.


In [ ]:
# Check traindata
for i in range(10): # lệnh !ls không hiểu python nên ko dùng '{}'.format i
    !ls trainingSet/{i}| wc -l

ls: cannot access 'trainingSet/0': No such file or directory
0
ls: cannot access 'trainingSet/1': No such file or directory
0
ls: cannot access 'trainingSet/2': No such file or directory
0
ls: cannot access 'trainingSet/3': No such file or directory
0
ls: cannot access 'trainingSet/4': No such file or directory
0
ls: cannot access 'trainingSet/5': No such file or directory
0
ls: cannot access 'trainingSet/6': No such file or directory
0
ls: cannot access 'trainingSet/7': No such file or directory
0
ls: cannot access 'trainingSet/8': No such file or directory
0
ls: cannot access 'trainingSet/9': No such file or directory
0


In [ ]:
# Check testdata
!ls testingSet/| wc -l

ls: cannot access 'testingSet/': No such file or directory
0


In [ ]:
!ls trainingSet/| wc -l

ls: cannot access 'trainingSet/': No such file or directory
0


# Preprocessing Data

In [ ]:
import tensorflow as tf
from pathlib import Path

In [ ]:
train_root = Path('trainingSetVNC/')
testing_root=Path('testingSetVNC/')
# convert to string type
filelist_ds= tf.data.Dataset.list_files(str(train_root/'*/*')) # 2 level scan (train_root to folders 0 - 9 then access each one)
# 9 random elements
for file in filelist_ds.take(4):
  print(file) #filename path in tf.data

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, '/')
  #part[0] = train
  #part[1] = 0/1.../9
  #part[2] = filename
  if parts[-2] == "0":
    labels=[0,0,0,0,0,0,0,0,0,1]
  elif parts[-2] == "1":
    labels=[1,0,0,0,0,0,0,0,0,0]
  elif parts[-2] == "2":
    labels=[0,1,0,0,0,0,0,0,0,0]
  elif parts[-2] == "3":
    labels=[0,0,1,0,0,0,0,0,0,0]
  elif parts[-2] == "4":
    labels=[0,0,0,1,0,0,0,0,0,0]
  elif parts[-2] == "5":
    labels=[0,0,0,0,1,0,0,0,0,0]
  elif parts[-2] == "6":
    labels=[0,0,0,0,0,1,0,0,0,0]
  elif parts[-2] == "7":
    labels=[0,0,0,0,0,0,1,0,0,0]
  elif parts[-2] == "8":
    labels=[0,0,0,0,0,0,0,1,0,0]
  elif parts[-2] == "9":
    labels=[0,0,0,0,0,0,0,0,1,0]
  else: 
    labels=[0,0,0,0,0,0,0,0,0,0]
    #return labels value but have to convert to tensor becase string we use is python not tf.tensor
  return tf.convert_to_tensor(labels) 

In [ ]:
# Test
for file in filelist_ds.take(9):
  print('filename',file.numpy().decode('utf-8')) # utf8 bỏ tf.tensor
  print('label:',get_label(file).numpy())

In [ ]:
from tensorflow._api.v2 import image
# Preprocessing function
img_height = 180
img_width = 180
def preprocessing(file_path): #data we have just a file list but data for train model must be pixel value
  # Read file
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img,channels=3) # 1 is gray, 3 is color; we use image function of tf.image ; jpeg because jpg is our flie
  # Transform
    # Conver from uint 8 to float 32 and Normalize value to [0,1]
  img = tf.image.convert_image_dtype(img,tf.float32)
    # Resize
  img = tf.image.resize(img,[img_width,img_height])
    # Get image label
  label = get_label(file_path)
  # Return
  return img, label

In [ ]:
ds_size = 25000 #(shuffle size) 
# Set parameter
batch_size = 128
# Build data
train_ds = filelist_ds.shuffle(ds_size) # Shuffling the input and it doesn't have labels
train_ds = train_ds.map(preprocessing,num_parallel_calls=tf.data.AUTOTUNE) # Process each element + get labels => train_ds = pixel val and labels
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(8) # chia luồn data

# Build Model

In [ ]:
# Build model 
# Load VGG model
base_model = tf.keras.applications.VGG16(
    weights='imagenet',
    input_shape=(180,180,3),
    include_top = False, # remove classifier output
)
base_model.trainable = False # No retrain
# Have 10 class (0,..9)
number_of_classes =10  
# Create model with functional API
inputs = tf.keras.Input(shape=(180,180,3))
x = base_model(inputs, training=False)
# Add more declaration 
x = tf.keras.layers.GlobalAveragePooling2D()(x) # include_top is remove (output remove) -> create output
# Outputs Dense 10 classes
outputs = tf.keras.layers.Dense(number_of_classes,activation='softmax')(x)
# Creat model
model = tf.keras.Model(inputs,outputs)
# Show model
tf.keras.utils.plot_model(model,'MNIST model.png',show_shapes=True)

# Compile and training model

In [ ]:
# Compile model
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics=["accuracy"] #caculate accuracy on each batch
)

In [ ]:
# Train model
model.fit(train_ds, epochs = 10)

In [ ]:
model.save('VietnamCurrentANNmodel.h5')